# MERGE016
Best of Each Type as of July 24

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import os
from sklearn.metrics import mean_absolute_error

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

In [2]:
types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC','3JHH','3JHC', '3JHN']
oof = pd.read_csv('../input/train.csv')
sub = pd.read_csv('../input/test.csv')
sub_final = pd.DataFrame()
oof_final = pd.DataFrame()
sub = sub.drop(['molecule_name','atom_index_0','atom_index_1'], axis=1)
oof = oof.drop(['molecule_name','atom_index_0','atom_index_1'], axis=1)

for t in types:
    subt = sub.loc[sub['type'] == t].copy()
    ooft = oof.loc[oof['type'] == t].copy()
    print(f'====== Running for type {t} ========')
    for file in os.listdir(f'../type_results/{t}/'):
        if '3folds' in file:
    #        print(file)
            if 'oof' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                ooft = ooft.merge(df[['id','oof_preds']] \
                              .rename(columns={'oof_preds': file[:4]}),
                          on='id')
            if 'sub' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                subt = subt.merge(df[['id','scalar_coupling_constant']] \
                              .rename(columns={'scalar_coupling_constant': file[:4]}),
                          on='id')
                print(file)
    #             df = pd.read_parquet(f'../type_results/{t}/{file}')
    #             sub = pd.concat([sub, df])
    sub_final = pd.concat([subt, sub_final], sort=False)
    oof_final = pd.concat([ooft, oof_final], sort=False)

====== Running for type 1JHC ========
M053_0725_0821_1JHC_sub_0.4215MAE_-0.8639LMAE_catboost_3folds_500000iter_0.1lr.parquet
M048_0718_1745_1JHC_sub_0.4379MAE_-0.8258LMAE_catboost_3folds_500000iter_0.1lr.parquet
M055_0726_2356_1JHC_sub_0.4214MAE_-0.8641LMAE_catboost_3folds_500000iter_0.1lr.parquet
====== Running for type 2JHH ========
M052_0723_1745_2JHH_sub_0.1001MAE_-2.301LMAE_catboost_3folds_500000iter_0.01lr.parquet
M054_0726_1820_2JHH_sub_0.1199MAE_-2.121LMAE_catboost_3folds_10000iter_0.1lr.parquet
M048_0717_2239_2JHH_sub_0.1020MAE_-2.282LMAE_catboost_3folds_500000iter_0.1lr.parquet
====== Running for type 1JHN ========
M049_0719_2005_1JHN_sub_0.2565MAE_-1.361LMAE_xgboost_3folds_100000iter_0.005lr.parquet
M053_0725_0821_1JHN_sub_0.2821MAE_-1.265LMAE_catboost_3folds_500000iter_0.1lr.parquet
M047_0714_2226_1JHN_submission_0.3276MAE_-1.116LMAE_catboost_3folds_500000iter_0.1lr.parquet
====== Running for type 2JHN ========
M053_0725_0821_2JHN_sub_0.1202MAE_-2.118LMAE_catboost_3folds_50

In [3]:
oof_final.head()

,id,type,scalar_coupling_constant,M047,M052,M050,M049,M055,M053,M048,M054
0,73,3JHN,0.880802,1.217714,0.907007,0.871911,NaN,NaN,NaN,NaN,NaN
1,78,3JHN,0.880957,1.199598,0.908139,0.873156,NaN,NaN,NaN,NaN,NaN
2,82,3JHN,0.880871,1.196354,0.863326,0.876223,NaN,NaN,NaN,NaN,NaN
3,213,3JHN,-0.052074,0.028468,0.037855,-0.135423,NaN,NaN,NaN,NaN,NaN
4,218,3JHN,1.611320,2.005109,1.819162,2.285958,NaN,NaN,NaN,NaN,NaN


In [11]:
mean_absolute_error(d['scalar_coupling_constant'],d[col])

0.4503699367015481

True

In [23]:
for i, d in oof_final.groupby('type'):
    best_lmae = 0
    for col in [x for x in oof_final.columns if 'M' in x]:
        if d[col].isna().any():
            continue
        lmae = np.log(mean_absolute_error(d['scalar_coupling_constant'],d[col]))
        print(f'{col}    {lmae:0.5f}')
        if lmae < best_lmae:
            oof_final.loc[oof_final['type'] == i, 'oof_final'] = d[col]
            sub_final.loc[sub_final['type'] == i, 'sub_final'] = sub_final.loc[sub_final['type'] == i][col]

M047    -0.79769
M049    -0.79101
M048    -0.82582
M047    -1.11607
M049    -1.36075
M050    -1.09335
M048    -1.08867
M047    -1.75030
M049    -1.72309
M048    -1.73679
M047    -2.20947
M049    -2.15493
M050    -2.12851
M048    -2.28249
M047    -2.12040
M049    -2.11730
M050    -2.08998
M048    -2.08225
M047    -1.61178
M049    -1.60615
M047    -2.09955
M049    -2.09766
M050    -2.02550
M048    -2.08565
M047    -2.35605
M049    -2.15576
M052    -2.39002
M050    -2.38316
M048    -2.32126


In [22]:
group_mean_log_mae(oof_final['scalar_coupling_constant'],
                   oof_final['oof_final'],
                   oof_final['type'])

-1.7536361788922883

# Compare with Good Sub

In [18]:
sub_good = pd.read_csv('../submissions/BLEND008_submission_-1.7087CV_Jul14best.csv')

In [19]:
sub['good_scc'] = sub_good['scalar_coupling_constant']

In [20]:
sub[['scalar_coupling_constant','good_scc']].corr()

,scalar_coupling_constant,good_scc
scalar_coupling_constant,1.000000,0.999987
good_scc,0.999987,1.000000


In [21]:
sub = sub.sort_values('id').reset_index(drop=True)
oof = oof.sort_values('id').reset_index(drop=True)

In [22]:
# Save Results
sub[['id','scalar_coupling_constant']].to_csv('../submissions/MERGE014_sub_M046_M047_-1.7313CV.csv', index=False)
oof.to_csv('../oof/MERGE014_oof_M046_M047_-1.7313CV.csv', index=False)